In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GoEmotions using PRADO



## Setup

The seq_flow_lite library has been written with the assumption that tensorflow 2.10.0 will be used.  It may be necessary to restart the runtime after installing the correct version of Tensorflow.

In [2]:
!pip install -q tensorflow==2.10.0

Update CuDNN.  The version installed on the Colab machines does not play well with Tensorflow 2.10.0.

In [3]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


### Install the TensorFlow Datasets pip package

`tensorflow_datasets` is a set of collection of datasets that includes the GoEmotions dataset. We install it with pip.

In [4]:
!pip install tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Install the Sequence Projection Models package

Install Bazel: This will allow us to build custom TensorFlow ops used by the PRADO architecture.

In [5]:
!sudo apt install curl gnupg
!curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update
!sudo apt install bazel

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.21).
gnupg is already the newest version (2.2.4-1ubuntu1.6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4714  100  4714    0     0  14504      0 --:--:-- --:--:-- --:--:-- 14504
OK
deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8
Hit:1 https://storage.googleapis.com/bazel-apt stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get

Install the library:
* `seq_flow_lite` includes the PRADO architecture and custom ops.
* We download the code from GitHub, and then build and install the TF and TFLite ops used by the model.


In [6]:
!git clone https://www.github.com/tensorflow/models
!models/research/seq_flow_lite/demo/colab/setup_workspace.sh
!pip install models/research/seq_flow_lite
!rm -rf models/research/seq_flow_lite/tf_ops
!rm -rf models/research/seq_flow_lite/tflite_ops

fatal: destination path 'models' already exists and is not an empty directory.
mv: cannot stat 'setup.py': No such file or directory
touch: cannot touch '../../tf_ops/__init__.py': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./models/research/seq_flow_lite
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


## Training an Emotion Prediction Model

* First, we load the GoEmotions data from TFDS.
* Next, we prepare the PRADO model for training. We set up the model configuration, including hyperparameters and labels. We also prepare the dataset, which involves projecting the inputs from the dataset, and passing the projections to the model.  This is needed because a model training on TPU can not handle string inputs.
* Finally, we train and evaluate the model and produce model-level and per-label metrics.

In [7]:
%cd models/research/seq_flow_lite

/content/models/research/seq_flow_lite


* Import the Tensorflow and Tensorflow Dataset libraries.

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

Load the data from TFDS:

In [9]:
ds = tfds.load('goemotions', split='train')

In [10]:
#function to subset the dataset to single label entries only
def remove_multilabels(ds):
    df = tfds.as_dataframe(ds)
    # df = df.drop('grief', axis=1) # drop grief column 
    col_names = [c for c in df.columns if c!="comment_text" ]
    df['sum'] = df[col_names].sum(axis=1)
    df = df[df['sum']==1]
    df = df.drop('sum', axis=1)
    ds = tf.data.Dataset.from_tensor_slices(dict(df))
    print()
    return ds
    # return df

#subset data to single label entries only
# ds_train = remove_multilabels(ds, 'train')
# ds_valid = remove_multilabels(ds, 'validation')
# ds_test = remove_multilabels(ds, 'test')


In [11]:

# Creating TF tensors
# train_tensor = tf.data.Dataset.from_tensor_slices((train, y_train)).shuffle(len(train)).batch(batchsize)
# val_tensor = tf.data.Dataset.from_tensor_slices((val, y_valid)).shuffle(len(val)).batch(batchsize)
# test_tensor = tf.data.Dataset.from_tensor_slices((test, y_test)).shuffle(len(test)).batch(batchsize)


Print 5 sample data elements from the dataset:

In [12]:
for element in ds.take(5):
  print(element)

{'admiration': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'amusement': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'anger': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'annoyance': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'approval': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'caring': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'comment_text': <tf.Tensor: shape=(), dtype=string, numpy=b"It's just wholesome content, from questionable sources">, 'confusion': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'curiosity': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'desire': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'disappointment': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'disapproval': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'disgust': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'embarrassment': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'excitement': <tf.Tensor: shape=(), dtype=bool, numpy=False>, 'fear': <tf.Tensor: shape=(

### The model: PRADO

Prepare GoEmotions and PRADO for training.

Prepare the model configuration:
* Enumerate the labels expected to be found in the GoEmotions dataset.
* Prepare the `MODEL_CONFIG` dictionary which includes training parameters for the model. See sample configs for the PRADO model [here](https://github.com/tensorflow/models/tree/master/research/seq_flow_lite/configs).

In [13]:
LABELS = [
    'admiration',
    'amusement',
    'anger',
    'annoyance',
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]

# Model training parameters.
CONFIG = {
    'name': 'models.prado',
    'batch_size': 1024,
    'train_steps': 50000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}

# BERT CONFIG for comparison
CONFIG2 = {
    'name': 'models.prado',
    'batch_size': 32,
    # 'train_steps': 10000,
    'learning_rate': 0.00005,
    # 'learning_rate_decay_steps': 340,
    # 'learning_rate_decay_rate': 0.7,
}


# Limits the amount of logging output produced by the training run, in order to
# avoid browser slowdowns.
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 25)

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': True,  
    'quantize': False,
    'max_seq_len': 128,  
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
    'doc_size_levels': 0,
    'add_bos_tag': False,
    'add_eos_tag': False,
    'pre_logits_fc_layers': [],
    'text_distortion_probability': 0.0,
}

CONFIG['model_config'] = MODEL_CONFIG

## BERT Model Preprocessing Function

In [14]:
#SUBSET
# Building a preprocessing function to clean text
def preprocess_corpus(x):

  # Adding a space between words and punctation
  x = re.sub( r'([a-zA-Z\[\]])([,;.!?])', r'\1 \2', x)
  x = re.sub( r'([,;.!?])([a-zA-Z\[\]])', r'\1 \2', x)
  
  # Demojize
  x = emoji.demojize(x)
  
  # Expand contraction
  x = contractions.fix(x)
  
  # Lower
  x = x.lower()

  # #correct some acronyms/typos/abbreviations  
  # x = re.sub(r"lmao", "laughing my ass off", x)  
  # x = re.sub(r"amirite", "am i right", x)
  # x = re.sub(r"\b(tho)\b", "though", x)
  # x = re.sub(r"\b(ikr)\b", "i know right", x)
  # x = re.sub(r"\b(ya|u)\b", "you", x)
  # x = re.sub(r"\b(eu)\b", "europe", x)
  # x = re.sub(r"\b(da)\b", "the", x)
  # x = re.sub(r"\b(dat)\b", "that", x)
  # x = re.sub(r"\b(dats)\b", "that is", x)
  # x = re.sub(r"\b(cuz)\b", "because", x)
  # x = re.sub(r"\b(fkn)\b", "fucking", x)
  # x = re.sub(r"\b(tbh)\b", "to be honest", x)
  # x = re.sub(r"\b(tbf)\b", "to be fair", x)
  # x = re.sub(r"faux pas", "mistake", x)
  # x = re.sub(r"\b(btw)\b", "by the way", x)
  # x = re.sub(r"\b(bs)\b", "bullshit", x)
  # x = re.sub(r"\b(kinda)\b", "kind of", x)
  # x = re.sub(r"\b(bruh)\b", "bro", x)
  # x = re.sub(r"\b(w/e)\b", "whatever", x)
  # x = re.sub(r"\b(w/)\b", "with", x)
  # x = re.sub(r"\b(w/o)\b", "without", x)
  # x = re.sub(r"\b(doj)\b", "department of justice", x)
  
  # #replace some words with multiple occurences of a letter, example "coooool" turns into --> cool
  # x = re.sub(r"\b(j+e{2,}z+e*)\b", "jeez", x)
  # x = re.sub(r"\b(co+l+)\b", "cool", x)
  # x = re.sub(r"\b(g+o+a+l+)\b", "goal", x)
  # x = re.sub(r"\b(s+h+i+t+)\b", "shit", x)
  # x = re.sub(r"\b(o+m+g+)\b", "omg", x)
  # x = re.sub(r"\b(w+t+f+)\b", "wtf", x)
  # x = re.sub(r"\b(w+h+a+t+)\b", "what", x)
  # x = re.sub(r"\b(y+e+y+|y+a+y+|y+e+a+h+)\b", "yeah", x)
  # x = re.sub(r"\b(w+o+w+)\b", "wow", x)
  # x = re.sub(r"\b(w+h+y+)\b", "why", x)
  # x = re.sub(r"\b(s+o+)\b", "so", x)
  # x = re.sub(r"\b(f)\b", "fuck", x)
  # x = re.sub(r"\b(w+h+o+p+s+)\b", "whoops", x)
  # x = re.sub(r"\b(ofc)\b", "of course", x)
  # x = re.sub(r"\b(the us)\b", "usa", x)
  # x = re.sub(r"\b(gf)\b", "girlfriend", x)
  # x = re.sub(r"\b(hr)\b", "human ressources", x)
  # x = re.sub(r"\b(mh)\b", "mental health", x)
  # x = re.sub(r"\b(idk)\b", "i do not know", x)
  # x = re.sub(r"\b(gotcha)\b", "i got you", x)
  # x = re.sub(r"\b(y+e+p+)\b", "yes", x)
  # x = re.sub(r"\b(a*ha+h[ha]*|a*ha +h[ha]*)\b", "haha", x)
  # x = re.sub(r"\b(o?l+o+l+[ol]*)\b", "lol", x)
  # x = re.sub(r"\b(o*ho+h[ho]*|o*ho +h[ho]*)\b", "ohoh", x)
  # x = re.sub(r"\b(o+h+)\b", "oh", x)
  # x = re.sub(r"\b(a+h+)\b", "ah", x)
  # x = re.sub(r"\b(u+h+)\b", "uh", x)

  # # Handling emojis
  x = re.sub(r"<3", " love_heart ", x)
  x = re.sub(r"xd", " smiling_face_with_open_mouth_and_tightly_closed_eyes ", x)
  x = re.sub(r":\)", " smiling_face ", x)
  x = re.sub(r"^_^", " smiling_face ", x)
  x = re.sub(r"\*_\*", " star_struck ", x)
  x = re.sub(r":\(", " frowning_face ", x)
  x = re.sub(r":\^\(", " frowning_face ", x)
  x = re.sub(r";\(", " frowning_face ", x)
  x = re.sub(r":\/",  " confused_face", x)
  x = re.sub(r";\)",  " wink", x)
  x = re.sub(r">__<",  " unamused ", x)
  x = re.sub(r"\b([xo]+x*)\b", " xoxo ", x)
  x = re.sub(r"\b(n+a+h+)\b", "nah", x)

  # # Handling special cases of text
  # x = re.sub(r"h a m b e r d e r s", "hamberders", x)
  # x = re.sub(r"b e n", "ben", x)
  # x = re.sub(r"s a t i r e", "satire", x)
  # x = re.sub(r"y i k e s", "yikes", x)
  # x = re.sub(r"s p o i l e r", "spoiler", x)
  # x = re.sub(r"thankyou", "thank you", x)
  # x = re.sub(r"a^r^o^o^o^o^o^o^o^n^d", "around", x)

  # Remove special characters and numbers replace by space + remove double space
  x = re.sub(r"\b([.]{3,})"," dots ", x)
  x = re.sub(r"[^A-Za-z!?_]+"," ", x)
  x = re.sub(r"\b([s])\b *","", x)
  x = re.sub(r" +"," ", x)
  x = x.strip()

  return x

Write a function that builds the datasets for the model.  It will load the data, handle batching, and generate projections for the input text.

In [15]:
from layers import base_layers
from layers import projection_layers

def build_dataset(mode, inspect=False):
  if mode == base_layers.TRAIN:
    split = 'train'
    count = None
  elif mode == base_layers.EVAL:
    split = 'test'
    count = 1
  else:
    raise ValueError('mode={}, must be TRAIN or EVAL'.format(mode))

  batch_size = CONFIG['batch_size']
  if inspect:
    batch_size = 1

  # Convert examples from their dataset format into the model format.
  def process_input(features):
    # Generate the projection for each comment_text input.  The final tensor 
    # will have the shape [batch_size, number of tokens, feature size].
    # Additionally, we generate a tensor containing the number of tokens for
    # each comment_text (seq_length).  This is needed because the projection
    # tensor is a full tensor, and we are not using EOS tokens.
    text = features['comment_text']
    text = tf.reshape(text, [batch_size])
    projection_layer = projection_layers.ProjectionLayer(MODEL_CONFIG, mode)
    projection, seq_length = projection_layer(text)

    # Convert the labels into an indicator tensor, using the LABELS indices.
    label = tf.stack([features[label] for label in LABELS], axis=-1)
    label = tf.cast(label, tf.float32)
    label = tf.reshape(label, [batch_size, len(LABELS)])

    model_features = ({'projection': projection, 'sequence_length': seq_length}, label)

    if inspect:
      model_features = (model_features[0], model_features[1], features)

    return model_features

  ds = tfds.load('goemotions', split=split)
  ds = remove_multilabels(ds)
  ds = ds.repeat(count=count)
  ds = ds.shuffle(buffer_size=batch_size * 2)
  ds = ds.batch(batch_size, drop_remainder=True)
  ds = ds.map(process_input,
              num_parallel_calls=tf.data.experimental.AUTOTUNE,
              deterministic=False)
  ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return ds

train_dataset = build_dataset(base_layers.TRAIN)
test_dataset = build_dataset(base_layers.EVAL)
inspect_dataset = build_dataset(base_layers.TRAIN, inspect=True)

In [16]:
train_dataset.take(1)

<TakeDataset element_spec=({'projection': TensorSpec(shape=(1024, 128, 512), dtype=tf.float32, name=None), 'sequence_length': TensorSpec(shape=(1024,), dtype=tf.float32, name=None)}, TensorSpec(shape=(1024, 28), dtype=tf.float32, name=None))>

Print a batch of examples in model format.  This will consist of:
* the projection tensors (projection and seq_length)
* the label tensor (second tuple value)

The projection tensor is a **[batch size, max_seq_length, feature_size]** floating point tensor.  The **[b, i]** vector is a feature vector of the **i**th token of the **b**th comment_text.  The rest of the tensor is zero-padded, and the
seq_length tensor indicates the number of features vectors for each comment_text.

The label tensor is an indicator tensor of the set of true labels for the example.

In [17]:
example = next(iter(train_dataset))
print("inputs = {}".format(example[0]))
print("labels = {}".format(example[1]))

inputs = {'projection': <tf.Tensor: shape=(1024, 128, 512), dtype=float32, numpy=
array([[[ 1.,  0.,  1., ...,  1., -1.,  0.],
        [-1.,  1.,  0., ...,  1., -1.,  1.],
        [ 0., -1.,  0., ...,  0., -1.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[-1.,  0.,  1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1.,  1.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0., -1.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  1.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0., -1.],
        [ 1., -1.,  0., ...,  0.,  1.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ...,

       [[ 0.,  1., -1., ...,  1.,  1.,  0.],
     

In this version of the dataset, the original example has been added as the third element of the tuple.

In [18]:
example = next(iter(inspect_dataset))
print("inputs = {}".format(example[0]))
print("labels = {}".format(example[1]))
print("original example = {}".format(example[2]))

inputs = {'projection': <tf.Tensor: shape=(1, 128, 512), dtype=float32, numpy=
array([[[ 1., -1.,  1., ...,  1.,  0., -1.],
        [ 0.,  1., -1., ..., -1., -1.,  0.],
        [-1.,  1.,  0., ..., -1., -1.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]], dtype=float32)>, 'sequence_length': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>}
labels = [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
original example = {'admiration': <tf.Tensor: shape=(1,), dtype=bool, numpy=array([ True])>, 'amusement': <tf.Tensor: shape=(1,), dtype=bool, numpy=array([False])>, 'anger': <tf.Tensor: shape=(1,), dtype=bool, numpy=array([False])>, 'annoyance': <tf.Tensor: shape=(1,), dtype=bool, numpy=array([False])>, 'approval': <tf.Tensor: shape=(1,), dtype=bool, numpy=array([False])>, 'caring': <tf.Tensor: shape=(1,), dtype=bool, numpy

### Train and Evaluate

First we define a function to build the model.  We vary the model inputs depending on task.  For training and evaluation, we'll take the projection and sequence length as inputs.  Otherwise, we'll take strings as inputs.

In [19]:
from models import prado

def build_model(mode):
  # First we define our inputs.
  inputs = []
  if mode == base_layers.TRAIN or mode == base_layers.EVAL:
    # For TRAIN and EVAL, we'll be getting dataset examples,
    # so we'll get projections and sequence_lengths.
    projection = tf.keras.Input(
        shape=(MODEL_CONFIG['max_seq_len'], MODEL_CONFIG['feature_size']),
        name='projection',
        dtype='float32')

    sequence_length = tf.keras.Input(
        shape=(), name='sequence_length', dtype='float32')
    inputs = [projection, sequence_length]
  else:
    # Otherwise, we get string inputs which we need to project.
    input = tf.keras.Input(shape=(), name='input', dtype='string')
    projection_layer = projection_layers.ProjectionLayer(MODEL_CONFIG, mode)
    projection, sequence_length = projection_layer(input)
    inputs = [input]

  # Next we add the model layer.
  model_layer = prado.Encoder(MODEL_CONFIG, mode)
  logits = model_layer(projection, sequence_length)

  # Finally we add an activation layer.
  if MODEL_CONFIG['multilabel']:
    activation = tf.keras.layers.Activation('sigmoid', name='predictions')
  else:
    activation = tf.keras.layers.Activation('softmax', name='predictions')
  predictions = activation(logits)

  model = tf.keras.Model(
      inputs=inputs,
      outputs=[predictions])
  
  return model


Train the model:

In [20]:
# Remove any previous training data.
!rm -rf model

model = build_model(base_layers.TRAIN)

# Create the optimizer.
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=CONFIG['learning_rate'],
    decay_rate=CONFIG['learning_rate_decay_rate'],
    decay_steps=CONFIG['learning_rate_decay_steps'],
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Define the loss function.
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

model.compile(optimizer=optimizer, loss=loss)

epochs = int(CONFIG['train_steps'] / CONFIG['save_checkpoints_steps'])
model.fit(
    x=train_dataset,
    epochs=epochs,
    validation_data=test_dataset,
    steps_per_epoch=CONFIG['save_checkpoints_steps'])

model.save_weights('model/model_checkpoint')

Epoch 1/25
2000/2000 [==============================] - 164s 80ms/step - loss: 0.5479 - val_loss: 0.4694
Epoch 2/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4501 - val_loss: 0.4434
Epoch 3/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4397 - val_loss: 0.4390
Epoch 4/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4385 - val_loss: 0.4413
Epoch 5/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4382 - val_loss: 0.4423
Epoch 6/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4382 - val_loss: 0.4416
Epoch 7/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4382 - val_loss: 0.4404
Epoch 8/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4381 - val_loss: 0.4409
Epoch 9/25
2000/2000 [==============================] - 159s 79ms/step - loss: 0.4382 - val_loss: 0.4395
Epoch 10/25
2000/2000 [==============================] 

Load a training checkpoint and evaluate:

In [21]:
model = build_model(base_layers.EVAL)

# Define metrics over each category.
metrics = []
for i, label in enumerate(LABELS):
  metric = tf.keras.metrics.Precision(
      thresholds=[0.5],
      class_id=i,
      name='precision@0.5/{}'.format(label))
  metrics.append(metric)
  metric = tf.keras.metrics.Recall(
      thresholds=[0.5],
      class_id=i,
      name='recall@0.5/{}'.format(label))
  metrics.append(metric)

# Define metrics over the entire task.
metric = tf.keras.metrics.Precision(thresholds=[0.5], name='precision@0.5/all')
metrics.append(metric)
metric = tf.keras.metrics.Recall(thresholds=[0.5], name='recall@0.5/all')
metrics.append(metric)

model.compile(metrics=metrics)
model.load_weights('model/model_checkpoint')
result = model.evaluate(x=test_dataset, return_dict=True)

4/4 [==============================] - 4s 74ms/step - loss: 0.0000e+00 - precision@0.5/admiration: 0.5000 - recall@0.5/admiration: 0.5357 - precision@0.5/amusement: 0.7114 - recall@0.5/amusement: 0.6386 - precision@0.5/anger: 0.4091 - recall@0.5/anger: 0.3879 - precision@0.5/annoyance: 0.0000e+00 - recall@0.5/annoyance: 0.0000e+00 - precision@0.5/approval: 0.0000e+00 - recall@0.5/approval: 0.0000e+00 - precision@0.5/caring: 0.0000e+00 - recall@0.5/caring: 0.0000e+00 - precision@0.5/confusion: 0.0000e+00 - recall@0.5/confusion: 0.0000e+00 - precision@0.5/curiosity: 0.5833 - recall@0.5/curiosity: 0.0443 - precision@0.5/desire: 0.4783 - recall@0.5/desire: 0.2391 - precision@0.5/disappointment: 0.0000e+00 - recall@0.5/disappointment: 0.0000e+00 - precision@0.5/disapproval: 1.0000 - recall@0.5/disapproval: 0.0061 - precision@0.5/disgust: 0.0000e+00 - recall@0.5/disgust: 0.0000e+00 - precision@0.5/embarrassment: 0.0000e+00 - recall@0.5/embarrassment: 0.0000e+00 - precision@0.5/excitement: 0.

Print evaluation metrics for the model, as well as per emotion label:

In [22]:
for label in LABELS:
  precision_key = 'precision@0.5/{}'.format(label)
  recall_key = 'recall@0.5/{}'.format(label)
  if precision_key in result and recall_key in result:
    print('{}: (precision@0.5: {}, recall@0.5: {})'.format(
        label, result[precision_key], result[recall_key]))
    
precision_key = 'precision@0.5/all'
recall_key = 'recall@0.5/all'
if precision_key in result and recall_key in result:
  print('all: (precision@0.5: {}, recall@0.5: {})'.format(
      result[precision_key], result[recall_key]))

admiration: (precision@0.5: 0.5, recall@0.5: 0.5357142686843872)
amusement: (precision@0.5: 0.7114093899726868, recall@0.5: 0.6385542154312134)
anger: (precision@0.5: 0.40909090638160706, recall@0.5: 0.38793104887008667)
annoyance: (precision@0.5: 0.0, recall@0.5: 0.0)
approval: (precision@0.5: 0.0, recall@0.5: 0.0)
caring: (precision@0.5: 0.0, recall@0.5: 0.0)
confusion: (precision@0.5: 0.0, recall@0.5: 0.0)
curiosity: (precision@0.5: 0.5833333134651184, recall@0.5: 0.04430379718542099)
desire: (precision@0.5: 0.47826087474823, recall@0.5: 0.239130437374115)
disappointment: (precision@0.5: 0.0, recall@0.5: 0.0)
disapproval: (precision@0.5: 1.0, recall@0.5: 0.0060606058686971664)
disgust: (precision@0.5: 0.0, recall@0.5: 0.0)
embarrassment: (precision@0.5: 0.0, recall@0.5: 0.0)
excitement: (precision@0.5: 0.6666666865348816, recall@0.5: 0.03773584961891174)
fear: (precision@0.5: 0.0, recall@0.5: 0.0)
gratitude: (precision@0.5: 0.9463414549827576, recall@0.5: 0.8398268222808838)
grief: 

In [23]:
(0.5421106815338135*0.385009765625)/(0.5421106815338135+0.385009765625)

0.2251249091525965

admiration: (precision@0.5: 0.7708333134651184, recall@0.5: 0.11858974397182465)  
amusement: (precision@0.5: 0.7603305578231812, recall@0.5: 0.5508981943130493)  
anger: (precision@0.5: 0.7058823704719543, recall@0.5: 0.104347825050354)  
annoyance: (precision@0.5: 0.0, recall@0.5: 0.0)  
approval: (precision@0.5: 1.0, recall@0.5: 0.009708737954497337)  
caring: (precision@0.5: 0.0, recall@0.5: 0.0)  
confusion: (precision@0.5: 0.0, recall@0.5: 0.0)  
curiosity: (precision@0.5: 0.0, recall@0.5: 0.0)  
desire: (precision@0.5: 0.4761904776096344, recall@0.5: 0.19230769574642181)  
disappointment: (precision@0.5: 0.0, recall@0.5: 0.0)  
disapproval: (precision@0.5: 0.0, recall@0.5: 0.0)  
disgust: (precision@0.5: 0.0, recall@0.5: 0.0)  
embarrassment: (precision@0.5: 0.0, recall@0.5: 0.0)  
excitement: (precision@0.5: 0.0, recall@0.5: 0.0)  
fear: (precision@0.5: 0.0, recall@0.5: 0.0)  
gratitude: (precision@0.5: 0.963350772857666, recall@0.5: 0.8288288116455078)  
grief: (precision@0.5: 0.0, recall@0.5: 0.0)  
joy: (precision@0.5: 0.6399999856948853, recall@0.5: 0.20253165066242218)  
love: (precision@0.5: 0.8571428656578064, recall@0.5: 0.5416666865348816)  
nervousness: (precision@0.5: 0.0, recall@0.5: 0.0)  
optimism: (precision@0.5: 0.7647058963775635, recall@0.5: 0.4193548262119293)  
pride: (precision@0.5: 0.0, recall@0.5: 0.0)  
realization: (precision@0.5: 0.0, recall@0.5: 0.0)  
relief: (precision@0.5: 0.0, recall@0.5: 0.0)  
remorse: (precision@0.5: 0.5142857432365417, recall@0.5: 0.5142857432365417)  
sadness: (precision@0.5: 0.0, recall@0.5: 0.0)  
surprise: (precision@0.5: 0.0, recall@0.5: 0.0)  
neutral: (precision@0.5: 0.800000011920929, recall@0.5: 0.002805049065500498)  
all: (precision@0.5: 0.8092105388641357, recall@0.5: 0.1201171875)  

## Suggest Emojis using an Emotion Prediction model

In this section, we apply the Emotion Prediction model trained above to suggest emojis relevant to input text.

Refer to our [GoEmotions Model Card](https://github.com/google-research/google-research/blob/master/goemotions/goemotions_model_card.pdf) for additional uses of the model and considerations and limitations for using the GoEmotions data.

Map each emotion label to a relevant emoji:
* Emotions are subtle and multi-faceted. In many cases, no one emoji can truely capture the full complexity of the human experience behind each emotion. 
* For the purpose of this exercise, we will select an emoji that captures at least one facet that is conveyed by an emotion label.

In [25]:
EMOJI_MAP = {
    'admiration': '👏',
    'amusement': '😂',
    'anger': '😡',
    'annoyance': '😒',
    'approval': '👍',
    'caring': '🤗',
    'confusion': '😕',
    'curiosity': '🤔',
    'desire': '😍',
    'disappointment': '😞',
    'disapproval': '👎',
    'disgust': '🤮',
    'embarrassment': '😳',
    'excitement': '🤩',
    'fear': '😨',
    'gratitude': '🙏',
    'grief': '😢',
    'joy': '😃',
    'love': '❤️',
    'nervousness': '😬',
    'optimism': '🤞',
    'pride': '😌',
    'realization': '💡',
    'relief': '😅',
    'remorse': '',
    'sadness': '😞',
    'surprise': '😲',
    'neutral': '',
}

Select sample inputs:

In [28]:
PREDICT_TEXT = [
  b'Good for you!',
  b'Happy birthday!',
  b'I love you.',
  b'Who are you?',
  b'Do not talk so loud.'
]

Run inference for the selected examples:

In [29]:
import numpy as np

model = build_model(base_layers.PREDICT)
model.load_weights('model/model_checkpoint')

for text in PREDICT_TEXT:
  results = model.predict(x=[text])
  print('')
  print('{}:'.format(text))
  labels = np.flip(np.argsort(results[0]))
  for x in range(3):
    label = LABELS[labels[x]]
    label = EMOJI_MAP[label] if EMOJI_MAP[label] else label
    print('{}: {}'.format(label, results[0][labels[x]]))

1/1 [==============================] - 0s 327ms/step

b'Good for you!':
👏: 0.8067114353179932
😃: 0.40806859731674194
👍: 0.37184765934944153
1/1 [==============================] - 0s 34ms/step

b'Happy birthday!':
😃: 0.9798732995986938
🤩: 0.5317233800888062
😳: 0.3652009665966034
1/1 [==============================] - 0s 34ms/step

b'I love you.':
❤️: 0.9999984502792358
😡: 0.3247499167919159
🙏: 0.3211614787578583
1/1 [==============================] - 0s 35ms/step

b'Who are you?':
🤔: 0.5164777040481567
neutral: 0.4811791479587555
😡: 0.3884563744068146
1/1 [==============================] - 0s 38ms/step

b'Do not talk so loud.':
neutral: 0.42998552322387695
👎: 0.4268779754638672
😕: 0.36860185861587524
